In [8]:
import sys
import torch
from thop import profile
sys.path.append('../')
from networks.networks import get_network

from networks import T2T
import timm

In [24]:
config = {'architecture':'T2T-ViT_t-14',
          'pretrained':False,
          'n_classes':4
         }
img_size = 384

inp = torch.randn(1,3,img_size,img_size)
network = get_network(config, img_size)
Attention = T2T.models.transformer_block.Attention
Attention = timm.models.vision_transformer.Attention
def count_your_model(model, x, y):
    x = x[0]
    B, N, C = x.shape
    total_ops = B*N*N*(C+C-1) # Q*K = Attention
    total_ops +=   B*N*C*(C+C-1)# V*Attention
    model.total_ops += torch.DoubleTensor([int(total_ops)])

macs, params = profile(network, inputs=(inp, ), custom_ops={Attention: count_your_model})


from thop import clever_format
clever_format([macs, params], "%.3f")

adopt performer encoder for tokens-to-token
[WARN] Cannot find rule for <class 'torch.nn.modules.fold.Unfold'>. Treat it as zero Macs and zero Params.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.normalization.LayerNorm'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.activation.GELU'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.T2T.models.token_performer.Token_performer'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.T2T.models.t2t_vit.T2T_module'>. Treat it as zero Macs and zero Params.
[WARN] Cannot find rule for <class 'networks.T2T.models.transformer_block.Attention'>. Treat it as zero 

('12.695G', '21.058M')